In [1]:
import csv
f = open('dataset/trainset.csv', 'r')
# f = open('e2e-dataset/testset_w_refs.csv', 'r')
r = csv.reader(f)

cond = []
sentence = []
for line in r:
    cond.append(line[0])
    sentence.append(line[1])    
    
typ_list = {}
for k in range(1, len(cond)):
    cond_set=cond[k].split(',')
    for m in range(len(cond_set)):
        cond_set[m] = cond_set[m].strip()
        pos = cond_set[m].index('[')
        if cond_set[m][:pos] in typ_list.keys():
            typ_list[cond_set[m][:pos]].add(cond_set[m][pos+1:-1])
        else:            
            typ_list[cond_set[m][:pos]] = {cond_set[m][pos+1:-1]}
            
#     print(k, typ_list)    

In [3]:
cond[2106], sentence[2106]

('name[The Rice Boat], food[English], priceRange[less than £20], customer rating[low], area[riverside], familyFriendly[yes], near[Express by Holiday Inn]',
 'The family friendly The Rice Boat is located on the riverside, near the Express by Holiday Inn, serving English cuisine below £20 with a customer rating.')

In [ ]:
import pandas as pd
x=pd.read_csv('dataset/testset_w_refs.csv')
x

In [51]:
# x.columns, x['mr'][0], x['ref'][0]
x['mr'][0]

'name[Blue Spice], eatType[coffee shop], area[city centre]'

In [20]:
for k, v in typ_list.items():
    print(k,v)

name {'The Vaults', 'Aromi', 'Green Man', 'The Cambridge Blue', 'Loch Fyne', 'Bibimbap House', 'Taste of Cambridge', 'Blue Spice', 'Midsummer House', 'The Golden Curry', 'The Wrestlers', 'The Waterman', 'Alimentum', 'Zizzi', 'Fitzbillies', 'The Plough', 'The Twenty Two', 'The Punter', 'The Cricketers', 'The Rice Boat', 'The Phoenix', 'Cotto', 'The Mill', 'Wildwood', 'Clowns', 'The Eagle', 'The Olive Grove', 'Cocum', 'The Dumpling Tree', 'Strada', 'Browns Cambridge', 'Giraffe', 'Travellers Rest Beefeater', 'The Golden Palace'} 34
eatType {'coffee shop', 'pub', 'restaurant'} 3
priceRange {'£20-25', 'moderate', 'cheap', 'more than £30', 'high', 'less than £20'} 6
customer rating {'average', 'low', '3 out of 5', '1 out of 5', '5 out of 5', 'high'} 6
near {'Café Sicilia', 'Avalon', 'The Six Bells', 'Café Rouge', 'The Portland Arms', 'The Rice Boat', 'Burger King', 'Ranch', 'Rainbow Vegetarian Café', 'Crowne Plaza Hotel', 'Clare Hall', 'Café Adriatic', 'Café Brazil', 'The Bakers', 'The Sorre

## Dataloader

In [2]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [3]:
class e2eDataset(Dataset):
    def __init__(self, csv_file):
        """
        Args:
            csv_file (string): csv 파일의 경로
        """
        self.dataset = pd.read_csv(csv_file)
        self.columns = self.dataset.columns
        self.conditions = self.dataset[self.columns[0]]
        self.sentences = self.dataset[self.columns[1]]
        
        self.typ_list = {}
        for k in range(len(self.conditions)):
            cond_set = self.conditions[k].split(',')
            for m in range(len(cond_set)):
                cond_set[m] = cond_set[m].strip()
                pos = cond_set[m].index('[')
                if cond_set[m][:pos] in self.typ_list.keys():
                    self.typ_list[cond_set[m][:pos]].add(cond_set[m][pos+1:-1])
                else:            
                    self.typ_list[cond_set[m][:pos]] = {cond_set[m][pos+1:-1]}        

    def __len__(self):
        return len(self.conditions)

    def __getitem__(self, idx):
        cond = self.conditions[idx]
        sen = self.sentences[idx]

        return cond, sen

In [4]:
e2e_dataset = e2eDataset(csv_file='dataset/trainset.csv')

In [5]:
e2e_dataset[0], e2e_dataset.typ_list

(('name[The Vaults], eatType[pub], priceRange[more than £30], customer rating[5 out of 5], near[Café Adriatic]',
  'The Vaults pub near Café Adriatic has a 5 star rating.  Prices start at £30.'),
 {'name': {'Alimentum',
   'Aromi',
   'Bibimbap House',
   'Blue Spice',
   'Browns Cambridge',
   'Clowns',
   'Cocum',
   'Cotto',
   'Fitzbillies',
   'Giraffe',
   'Green Man',
   'Loch Fyne',
   'Midsummer House',
   'Strada',
   'Taste of Cambridge',
   'The Cambridge Blue',
   'The Cricketers',
   'The Dumpling Tree',
   'The Eagle',
   'The Golden Curry',
   'The Golden Palace',
   'The Mill',
   'The Olive Grove',
   'The Phoenix',
   'The Plough',
   'The Punter',
   'The Rice Boat',
   'The Twenty Two',
   'The Vaults',
   'The Waterman',
   'The Wrestlers',
   'Travellers Rest Beefeater',
   'Wildwood',
   'Zizzi'},
  'eatType': {'coffee shop', 'pub', 'restaurant'},
  'priceRange': {'cheap',
   'high',
   'less than £20',
   'moderate',
   'more than £30',
   '£20-25'},
  'custome

In [12]:
dataloader = DataLoader(e2e_dataset, batch_size=1, shuffle=True, num_workers=4)
from model import *
my_model = mymodel()
my_model.eval()
my_model.load_state_dict(torch.load('./gen_model/final/model'))
print('ok')

In [29]:
x = [50258, 50259, 50263, 50260, 50261, 50264, 50265, 50262]
for i_batch, sample_batched in enumerate(dataloader):    
    cond = sample_batched[0][0]
    cond_set = cond.split(',')
    condition_string = ''
    
    for m in range(len(cond_set)):
        cond_set[m] = cond_set[m].strip()
        pos = cond_set[m].index('[')

        condition_string += '<' + cond_set[m][:pos] + '> '
    
    con_list = my_model.tokenizer.encode(condition_string.strip())
    
    out = []
    for k in range(len(con_list)):
        out.append(x.index(con_list[k]))
    print(out)
    assert out == sorted(out)
    if i_batch == 100:
        break

[0, 1, 2, 4, 6]
[0, 1, 2, 3, 5, 6, 7]
[0, 3, 4, 6, 7]
[0, 1, 2, 3, 5, 6]
[0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 4, 6, 7]
[0, 2, 4, 5, 6, 7]
[0, 2, 3, 4, 5, 6]
[0, 2, 6]
[0, 2, 3, 4, 7]
[0, 2, 5, 6, 7]
[0, 1, 2, 3, 5, 6]
[0, 2, 4, 5, 6]
[0, 2, 5, 6]
[0, 2, 4, 5]
[0, 2, 3, 4, 5, 7]
[0, 2, 4, 6]
[0, 1, 2, 3, 4, 5, 7]
[0, 1, 2, 3, 5, 6]
[0, 2, 3, 6]
[0, 2, 4, 5, 7]
[0, 1, 2, 3, 5, 6]
[0, 2, 3, 5, 6]
[0, 2, 3, 4, 6]
[0, 1, 3, 4, 5, 6, 7]
[0, 2, 3, 4]
[0, 2, 3, 6, 7]
[0, 1, 2, 3, 4, 5, 7]
[0, 1, 2, 3, 4, 6, 7]
[0, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 6]
[0, 2, 3, 4, 5]
[0, 1, 2, 3, 5, 6, 7]
[0, 2, 7]
[0, 1, 2, 3, 4, 5, 6, 7]
[0, 2, 4, 6]
[0, 1, 3, 5, 6]
[0, 1, 2, 3, 5, 6]
[0, 1, 2, 3, 5]
[0, 2, 3, 4, 6, 7]
[0, 2, 4, 5, 6]
[0, 1, 2, 4, 6]
[0, 1, 3, 4, 5, 6]
[0, 2, 3, 5, 7]
[0, 2, 4, 5, 6, 7]
[0, 1, 3, 4, 5, 6]
[0, 1, 2, 4, 5]
[0, 2, 3, 5, 6]
[0, 2, 3, 4, 5, 6, 7]
[0, 1, 4, 6, 7]
[0, 3, 4]
[0, 3, 6, 7]
[0, 2, 4, 5, 7]
[0, 1, 2, 4, 5, 7]
[0, 3, 5, 6]
[0, 2, 6]
[0, 2, 4, 5, 7]
[0, 1, 2, 3, 4, 6, 7]
[0, 1, 2, 3,

In [27]:
out, sorted(out)

([1, 6, 3, 4, 5], [1, 3, 4, 5, 6])

In [25]:
out=[1,6,3,4,5]

## Half dataset

In [35]:
import pandas as pd
import random
from torch.utils.data import Dataset, DataLoader
class e2eDataset(Dataset):
    def __init__(self, csv_file1, csv_file2):
        """
        Args:
            csv_file (string): csv 파일의 경로
        """
        self.dataset1 = pd.read_csv(csv_file1)
        self.dataset2 = pd.read_csv(csv_file2)
        
        self.columns1 = self.dataset1.columns
        self.columns2 = self.dataset2.columns
        
        self.conditions = list(self.dataset1[self.columns1[0]]) + list(self.dataset2[self.columns2[0]])
        self.sentences = list(self.dataset1[self.columns1[1]]) + list(self.dataset2[self.columns2[1]])
        
        self.typ_list = {}
        for k in range(len(self.conditions)):
            cond_set = self.conditions[k].split(',')
            for m in range(len(cond_set)):
                cond_set[m] = cond_set[m].strip()
                pos = cond_set[m].index('[')
                if cond_set[m][:pos] in self.typ_list.keys():
                    self.typ_list[cond_set[m][:pos]].add(cond_set[m][pos+1:-1])
                else:            
                    self.typ_list[cond_set[m][:pos]] = {cond_set[m][pos+1:-1]}
                    
        k = 0
        sample_order = []
        for _ in range(len(self.conditions)):
            sample_order.append(k)
            k += 1
        ran_num = random.sample(sample_order, int(len(self.conditions)/2))
        self.conditions_sample = [self.conditions[t] for t in ran_num]
        self.sentences_sample = [self.sentences[t] for t in ran_num]                  

    def __len__(self):
        return len(self.conditions_sample)

    def __getitem__(self, idx):
        cond = self.conditions_sample[idx]
        cond_set = cond.split(',')
        condition_string = ''
        for m in range(len(cond_set)):
            cond_set[m] = cond_set[m].strip()
            pos = cond_set[m].index('[')
            
            condition_string += '<' + cond_set[m][:pos] + '>' + cond_set[m][pos+1:-1] + ' '
        
        sen = self.sentences_sample[idx]

        return sen

In [36]:
e2e_dataset = e2eDataset(csv_file1='dataset/trainset.csv', csv_file2='dataset/devset.csv')
dataloader = DataLoader(e2e_dataset, batch_size=1, shuffle=True, num_workers=4)    

In [38]:
len(e2e_dataset.conditions), len(e2e_dataset.conditions_sample), len(e2e_dataset)

(46733, 23366, 23366)

In [ ]:
import random
x1 = [11,22,33,44,55,66]
x2 = [111,222,333,444,555,666]
k = 0
li = []
for _ in range(len(x1)):
    li.append(k)
    k += 1
ran_num = random.sample(li, int(len(x1)/2))
sample_x1 = [x1[t] for t in ran_num]
sample_x2 = [x2[t] for t in ran_num]
sample_x1, sample_x2

## Percentage check

In [8]:
import csv
import pandas as pd
# csv_file="/data/private/E2E/dataset/testset_w_refs.csv"
csv_file="/data/private/E2E/dataset/trainset.csv"
dataset = pd.read_csv(csv_file)
columns = dataset.columns
conditions = dataset[columns[0]]
refs = dataset[columns[1]]

incomplte_num = 0
total_value_num = 0
not_include_count  = 0
for ind in range(len(conditions)):
    cond_dict = {}
    cond_set = conditions[ind].split(',')
    
    for m in range(len(cond_set)):
        cond_set[m] = cond_set[m].strip()
        pos = cond_set[m].index('[')
        cond_dict[cond_set[m][:pos]] = (cond_set[m][pos+1:-1])    
        
    num = len(cond_dict)
    total_value_num += num
    
    sentence_temp = refs[ind]
    
    for k, v in cond_dict.items():
        if k != 'familyFriendly':
            if v.lower() not in sentence_temp.strip().lower():
                not_include_count += 1
            
    for k, v in cond_dict.items():
        if k != 'familyFriendly':
            if v.lower() not in sentence_temp.strip().lower():
                incomplte_num += 1
                break
                
print(not_include_count/total_value_num)
print(incomplte_num/len(conditions))

0.15221699281644785
0.474525094505599


In [6]:
# train
0.15221699281644785
0.474525094505599

# test
0.11579858963256216
0.4528020455998295

32332